In [0]:
from sklearn.model_selection import train_test_split
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
from datetime import datetime

In [0]:
!pip install bert-tensorflow

In [0]:
import bert
from bert import run_classifier
from bert import optimization
from bert import tokenization

In [0]:
import numpy as np
import nltk
from nltk import FreqDist
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import collections
from nltk.stem import WordNetLemmatizer
from autocorrect import spell
from googletrans import Translator
import re
import spacy
from spacy import displacy
from collections import Counter
import en_core_web_sm
import string
import math

In [0]:
df_raw = pd.read_excel("data/data_project_6371_2019_data_to_date_Sales SEPT 12.xlsx", sheet_name = "Sheet1")

In [0]:
pd.set_option('display.max_columns', None)

In [0]:
df_copy = df_raw.copy()

## Clean Dataset (Dont run it again)

In [0]:
cleaneddf = pd.read_csv("data/cleaneddf.tsv",sep="\t",header=None)

In [0]:
cleaneddf.columns = ['user_id','label','alpha','text']

In [0]:
cleaneddf = cleaneddf.reset_index(drop=True)

In [0]:
cleaneddf['user_id'] = cleaneddf.index
cleaneddf[cleaneddf['label']==0]

,user_id,label,alpha,text
33,33,0,a,Purolator always on time for pick-up & if it i...
75,75,0,a,Great Service and efficient Purolator Courier
99,99,0,a,pro
107,107,0,a,We use Purolator exclusively. It is pretty str...
161,161,0,a,GOOD AND FAST SERVICE
...,...,...,...,...
711,711,0,a,We are happy with service that we are getting....
712,712,0,a,Nothing awesome job.
713,713,0,a,Available when needed
714,714,0,a,"Nothing, her hands are tied."


In [0]:
#cleaneddf.to_csv("data/cleaneddf.tsv",sep="\t",index=False,header=False)

## Train Test Split

In [0]:
from sklearn.model_selection import train_test_split

In [0]:
X_train, X_test, y_train, y_test=train_test_split(cleaneddf, cleaneddf['label'],test_size=0.2)

In [0]:
X_train.to_csv("data/train.tsv",sep="\t",index=False,header=False)

In [0]:
X_test.to_csv("data/test.tsv",sep="\t",index=False,header=False)

## Data Preprocessing

In [0]:
df_sales_oe = df_copy[(df_copy['Sales7 OE'].isnull()!=True) & (df_copy['Sales7 OE']!=-66)][['MasterClientID','LanguagePreference','Sales7 OE','Sales2 NPS','Sales4 OSAT']]
df_sales_oe = df_sales_oe.replace('10 Very likely to recommend',10)
df_sales_oe = df_sales_oe.replace('10 Extremely Satisfied',10)
df_sales_oe = df_sales_oe.replace('0 Not at all likely to recommend',0)
df_sales_oe = df_sales_oe.replace('0 Not at all Satisfied',0)
df_sales_oe.rename(columns={'Sales7 OE':'comment', 'LanguagePreference':'language'}, inplace=True)
df_sales_oe
#df_sales_oe.set_index(['MasterClientID','TaskCompletionDate'], append = True, drop = False)

,MasterClientID,language,comment,Sales2 NPS,Sales4 OSAT
10,CUPR0190900009289271,English,Everything is good thanks .,10,10
12,CT00009801914,English,Reach out to us. We are never contacted by a Rep.,6,5
18,PUR1611404690MANITOB,English,Arrive on time and follow-up,7,4
21,CUPR0190900006402851,English,Turnover of Sales Rep is ridiculous. They are...,9,5
26,CUPR0190900003162268,English,I have nothing negative to say about our sales...,10,10
...,...,...,...,...,...
1117,CUPR0190900002831616,English,Not applicable.,10,5
1118,CT00009802003,English,"Nothing, her hands are tied.",9,9
1122,CUPR0190900005261332,English,CURRENTLY WORKING ON A ISSUE NOW.,10,9
1125,CUPR0190900002442075,English,See previous comment.,8,5


In [0]:
df_main_oe = df_copy[(df_copy['Sales3 OE'].isnull()!=True)& (df_copy['Sales3 OE']!=-66)][['MasterClientID','LanguagePreference','Sales3 OE','Sales2 NPS','Sales4 OSAT']]
df_main_oe = df_main_oe.replace('10 Very likely to recommend',10)
df_main_oe = df_main_oe.replace('10 Extremely Satisfied',10)
df_main_oe = df_main_oe.replace('0 Not at all likely to recommend',0)
df_main_oe = df_main_oe.replace('0 Not at all Satisfied',0)
df_main_oe.rename(columns={'Sales3 OE':'comment', 'LanguagePreference':'language'}, inplace=True)
df_main_oe

,MasterClientID,language,comment,Sales2 NPS,Sales4 OSAT
5,CUPR0190900002327268,French Canadian,GOOD AND FAST SERVICE,10,10
10,CUPR0190900009289271,English,GOOD SERVICE,10,10
11,CUPR0190900006204935,English,"good service, very quick pickup and delivery.",10,10
12,CT00009801914,English,Many express service shipments have not met th...,6,5
16,PUR-1124656961Epiroc,English,overall good service,8,10
...,...,...,...,...,...
1126,PUR446688177ECONOPAC,English,Courteous and Polite and on time,10,10
1129,CUPR0190900004019558,English,Purolator is reliable.,10,10
1134,CUPR0190900008683425,English,Most locations served,9,10
1135,PUR-1728507187Conuma,English,Everyone is knowledgeable in what their doing,10,10


In [0]:
df_copy = pd.concat([df_main_oe,df_sales_oe])

In [0]:
##REMOVING NaN
##df_copy = df_copy[(df_copy['Sales7 OE'].isnull()!=True) & (df_copy['Sales3 OE'].isnull()!=True) ]

In [0]:
##Translate from french to english
translator = Translator()

def translate_from_french(dataframe):
    df = dataframe.copy()
    series=[]
    for index, row in df.iterrows():
        if row['language'] == 'French Canadian':
            series.append(translator.translate(row['comment']).text)
        else:
            series.append(row['comment'])
    df['comment_modified'] = series
    return df
    #translator.translate('Mauvais service à la clientèle dans les points').text

#if df_copy['language']='French Canadian':
#    df_copy

In [0]:
df_copy = translate_from_french(df_copy)

In [0]:
df_copy.to_excel("data/modified_comments_data_translated_notnull.xlsx")

In [0]:
#df_negative = df_copy[(df_copy['Sales2 NPS']<7) & (df_copy['Sales4 OSAT']<7)] ##Negatives

In [0]:
#df_positive = df_copy[(df_copy['Sales2 NPS']>7) & (df_copy['Sales4 OSAT']>7)] ##Positives

In [0]:
#df_positive.shape[0]

In [0]:
#df_negative.shape[0]

In [0]:
#df1 = pd.DataFrame({'label':[1]*df_negative.shape[0],'alpha':['a']*df_negative.shape[0],'text':df_negative['comment_modified'].replace(r'\n',' ',regex=True)})

In [0]:
##df2 = pd.DataFrame({'label':[1]*df_negative.shape[0],'alpha':['a']*df_negative.shape[0],'text':df_negative['sales_comment_modified'].replace(r'\n',' ',regex=True)})

In [0]:
#df3 = pd.DataFrame({'label':[0]*df_positive.shape[0],'alpha':['a']*df_positive.shape[0],'text':df_positive['comment_modified'].replace(r'\n',' ',regex=True)})

In [0]:
##df4 = pd.DataFrame({'label':[0]*df_positive.shape[0],'alpha':['a']*df_positive.shape[0],'text':df_positive['sales_comment_modified'].replace(r'\n',' ',regex=True)})

In [0]:
#combined_bert_df = pd.concat([df1, df3], axis=0)

In [0]:
#combined_bert_df = combined_bert_df.reset_index(drop=True)

In [0]:
#combined_bert_df['user_id'] = combined_bert_df.index

In [0]:
#user_id = combined_bert_df['user_id']
#combined_bert_df.drop(labels=['user_id'], axis=1,inplace = True)
#combined_bert_df.insert(0, 'user_id', user_id)
#combined_bert_df

In [0]:
#combined_bert_df.to_csv("data/test.tsv",sep="\t",index=False,header=False)

## Train Model

Below, we'll set an output directory location to store our model output and checkpoints. This can be a local directory, in which case you'd set OUTPUT_DIR to the name of the directory you'd like to create. If you're running this code in Google's hosted Colab, the directory won't persist after the Colab session ends.

Alternatively, if you're a GCP user, you can store output in a GCP bucket. To do that, set a directory name in OUTPUT_DIR and the name of the GCP bucket in the BUCKET field.

Set DO_DELETE to rewrite the OUTPUT_DIR if it exists. Otherwise, Tensorflow will load existing model checkpoints from that directory (if they exist).

In [0]:
# Set the output directory for saving model file
# Optionally, set a GCP bucket location

OUTPUT_DIR = 'output-cleaned_dataset'#@param {type:"string"}
#@markdown Whether or not to clear/delete the directory and create a new one
DO_DELETE = False #@param {type:"boolean"}
#@markdown Set USE_BUCKET and BUCKET if you want to (optionally) store model output on GCP bucket.
USE_BUCKET = False #@param {type:"boolean"}
#BUCKET = 'BUCKET_NAME' #@param {type:"string"}

if USE_BUCKET:
  OUTPUT_DIR = 'gs://{}/{}'.format(BUCKET, OUTPUT_DIR)
  from google.colab import auth
  auth.authenticate_user()

if DO_DELETE:
  try:
    tf.gfile.DeleteRecursively(OUTPUT_DIR)
  except:
    # Doesn't matter if the directory didn't exist
    pass
tf.gfile.MakeDirs(OUTPUT_DIR)
print('***** Model output directory: {} *****'.format(OUTPUT_DIR))


***** Model output directory: output-cleaned_dataset *****


In [0]:
DATA_COLUMN = 'text'
LABEL_COLUMN = 'label'
# label_list is the list of labels, i.e. True, False or 0, 1 or 'dog', 'cat'
label_list = [0, 1]

In [0]:
train = X_train.copy()

In [0]:
train_InputExamples = train.apply(lambda x: bert.run_classifier.InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this example
                                                                   text_a = x[DATA_COLUMN], 
                                                                   text_b = None, 
                                                                   label = x[LABEL_COLUMN]), axis = 1)

In [0]:
train_InputExamples

478    <bert.run_classifier.InputExample object at 0x...
489    <bert.run_classifier.InputExample object at 0x...
492    <bert.run_classifier.InputExample object at 0x...
360    <bert.run_classifier.InputExample object at 0x...
616    <bert.run_classifier.InputExample object at 0x...
                             ...                        
355    <bert.run_classifier.InputExample object at 0x...
36     <bert.run_classifier.InputExample object at 0x...
658    <bert.run_classifier.InputExample object at 0x...
274    <bert.run_classifier.InputExample object at 0x...
16     <bert.run_classifier.InputExample object at 0x...
Length: 573, dtype: object

In [0]:
# This is a path to an uncased (all lowercase) version of BERT
BERT_MODEL_HUB = "https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1"

def create_tokenizer_from_hub_module():
  """Get the vocab file and casing info from the Hub module."""
  with tf.Graph().as_default():
    bert_module = hub.Module(BERT_MODEL_HUB)
    tokenization_info = bert_module(signature="tokenization_info", as_dict=True)
    with tf.Session() as sess:
      vocab_file, do_lower_case = sess.run([tokenization_info["vocab_file"],
                                            tokenization_info["do_lower_case"]])
      
  return bert.tokenization.FullTokenizer(
      vocab_file=vocab_file, do_lower_case=do_lower_case)

tokenizer = create_tokenizer_from_hub_module()

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [0]:
# We'll set sequences to be at most 128 tokens long.
MAX_SEQ_LENGTH = 128
# Convert our train and test features to InputFeatures that BERT understands.
train_features = bert.run_classifier.convert_examples_to_features(train_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)

INFO:tensorflow:Writing example 0 of 573


INFO:tensorflow:Writing example 0 of 573


INFO:tensorflow:*** Example ***


INFO:tensorflow:*** Example ***


INFO:tensorflow:guid: None


...

INFO:tensorflow:input_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1


INFO:tensorflow:segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


INFO:tensorflow:segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


INFO:tensorflow:label: 1 (id = 1)


INFO:tensorflow:label: 1 (id = 1)


In [0]:
train_features

In [0]:
def create_model(is_predicting, input_ids, input_mask, segment_ids, labels,
                 num_labels):
  """Creates a classification model."""

  bert_module = hub.Module(
      BERT_MODEL_HUB,
      trainable=True)
  bert_inputs = dict(
      input_ids=input_ids,
      input_mask=input_mask,
      segment_ids=segment_ids)
  bert_outputs = bert_module(
      inputs=bert_inputs,
      signature="tokens",
      as_dict=True)

  # Use "pooled_output" for classification tasks on an entire sentence.
  # Use "sequence_outputs" for token-level output.
  output_layer = bert_outputs["pooled_output"]

  hidden_size = output_layer.shape[-1].value

  # Create our own layer to tune for politeness data.
  output_weights = tf.get_variable(
      "output_weights", [num_labels, hidden_size],
      initializer=tf.truncated_normal_initializer(stddev=0.02))

  output_bias = tf.get_variable(
      "output_bias", [num_labels], initializer=tf.zeros_initializer())

  with tf.variable_scope("loss"):

    # Dropout helps prevent overfitting
    output_layer = tf.nn.dropout(output_layer, keep_prob=0.9)

    logits = tf.matmul(output_layer, output_weights, transpose_b=True)
    logits = tf.nn.bias_add(logits, output_bias)
    log_probs = tf.nn.log_softmax(logits, axis=-1)

    # Convert labels into one-hot encoding
    one_hot_labels = tf.one_hot(labels, depth=num_labels, dtype=tf.float32)

    predicted_labels = tf.squeeze(tf.argmax(log_probs, axis=-1, output_type=tf.int32))
    # If we're predicting, we want predicted labels and the probabiltiies.
    if is_predicting:
      return (predicted_labels, log_probs)

    # If we're train/eval, compute loss between predicted and actual label
    per_example_loss = -tf.reduce_sum(one_hot_labels * log_probs, axis=-1)
    loss = tf.reduce_mean(per_example_loss)
    return (loss, predicted_labels, log_probs)


In [0]:
import tensorflow as tf
tf.squeeze(tf.argmax((0.1,0.2), axis=-1, output_type=tf.int32))

<tf.Tensor 'Squeeze_1:0' shape=() dtype=int32>

In [0]:
# model_fn_builder actually creates our model function
# using the passed parameters for num_labels, learning_rate, etc.
def model_fn_builder(num_labels, learning_rate, num_train_steps,
                     num_warmup_steps):
  """Returns `model_fn` closure for TPUEstimator."""
  def model_fn(features, labels, mode, params):  # pylint: disable=unused-argument
    """The `model_fn` for TPUEstimator."""

    input_ids = features["input_ids"]
    input_mask = features["input_mask"]
    segment_ids = features["segment_ids"]
    label_ids = features["label_ids"]

    is_predicting = (mode == tf.estimator.ModeKeys.PREDICT)
    
    # TRAIN and EVAL
    if not is_predicting:

      (loss, predicted_labels, log_probs) = create_model(
        is_predicting, input_ids, input_mask, segment_ids, label_ids, num_labels)

      train_op = bert.optimization.create_optimizer(
          loss, learning_rate, num_train_steps, num_warmup_steps, use_tpu=False)

      # Calculate evaluation metrics. 
      def metric_fn(label_ids, predicted_labels):
        accuracy = tf.metrics.accuracy(label_ids, predicted_labels)
        f1_score = tf.contrib.metrics.f1_score(
            label_ids,
            predicted_labels)
        auc = tf.metrics.auc(
            label_ids,
            predicted_labels)
        recall = tf.metrics.recall(
            label_ids,
            predicted_labels)
        precision = tf.metrics.precision(
            label_ids,
            predicted_labels) 
        true_pos = tf.metrics.true_positives(
            label_ids,
            predicted_labels)
        true_neg = tf.metrics.true_negatives(
            label_ids,
            predicted_labels)   
        false_pos = tf.metrics.false_positives(
            label_ids,
            predicted_labels)  
        false_neg = tf.metrics.false_negatives(
            label_ids,
            predicted_labels)
        return {
            "eval_accuracy": accuracy,
            "f1_score": f1_score,
            "auc": auc,
            "precision": precision,
            "recall": recall,
            "true_positives": true_pos,
            "true_negatives": true_neg,
            "false_positives": false_pos,
            "false_negatives": false_neg
        }

      eval_metrics = metric_fn(label_ids, predicted_labels)

      if mode == tf.estimator.ModeKeys.TRAIN:
        return tf.estimator.EstimatorSpec(mode=mode,
          loss=loss,
          train_op=train_op)
      else:
          return tf.estimator.EstimatorSpec(mode=mode,
            loss=loss,
            eval_metric_ops=eval_metrics)
    else:
      (predicted_labels, log_probs) = create_model(
        is_predicting, input_ids, input_mask, segment_ids, label_ids, num_labels)

      predictions = {
          'probabilities': log_probs,
          'labels': predicted_labels
      }
      return tf.estimator.EstimatorSpec(mode, predictions=predictions)

  # Return the actual model function in the closure
  return model_fn


In [0]:
# Compute train and warmup steps from batch size
# These hyperparameters are copied from this colab notebook (https://colab.sandbox.google.com/github/tensorflow/tpu/blob/master/tools/colab/bert_finetuning_with_cloud_tpus.ipynb)
BATCH_SIZE = 32
LEARNING_RATE = 2e-5
NUM_TRAIN_EPOCHS = 3.0
# Warmup is a period of time where hte learning rate 
# is small and gradually increases--usually helps training.
WARMUP_PROPORTION = 0.1
# Model configs
SAVE_CHECKPOINTS_STEPS = 500
SAVE_SUMMARY_STEPS = 100

In [0]:
# Compute # train and warmup steps from batch size
num_train_steps = int(len(train_features) / BATCH_SIZE * NUM_TRAIN_EPOCHS)
num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)

In [0]:
# Specify outpit directory and number of checkpoint steps to save
run_config = tf.estimator.RunConfig(
    model_dir=OUTPUT_DIR,
    save_summary_steps=SAVE_SUMMARY_STEPS,
    save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS)

In [0]:
model_fn = model_fn_builder(
  num_labels=len(label_list),
  learning_rate=LEARNING_RATE,
  num_train_steps=num_train_steps,
  num_warmup_steps=num_warmup_steps)

estimator = tf.estimator.Estimator(
  model_fn=model_fn,
  config=run_config,
  params={"batch_size": BATCH_SIZE})


INFO:tensorflow:Using config: {'_model_dir': 'output-cleaned_dataset', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 500, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000001D8E5C9FC50>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


INFO:tensorflow:Using config: {'_model_dir': 'output-cleaned_dataset', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 500, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000001D8E5C9FC50>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [0]:
# Create an input function for training. drop_remainder = True for using TPUs.
train_input_fn = bert.run_classifier.input_fn_builder(
    features=train_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=True,
    drop_remainder=False)

In [0]:
print(f'Beginning Training!')
current_time = datetime.now()
estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)
print("Training took time ", datetime.now() - current_time)

Beginning Training!
Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.


Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.


INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


...

INFO:tensorflow:Saving checkpoints for 53 into output-cleaned_dataset\model.ckpt.


INFO:tensorflow:Saving checkpoints for 53 into output-cleaned_dataset\model.ckpt.


INFO:tensorflow:Loss for final step: 0.02079546.


INFO:tensorflow:Loss for final step: 0.02079546.


Training took time  0:19:07.329773


## TEST THE MODEL

In [0]:
test = X_test.copy()

In [0]:
test_InputExamples = test.apply(lambda x: bert.run_classifier.InputExample(guid=None, 
                                                                   text_a = x[DATA_COLUMN], 
                                                                   text_b = None, 
                                                                   label = x[LABEL_COLUMN]), axis = 1)

In [0]:
test_features = bert.run_classifier.convert_examples_to_features(test_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)

INFO:tensorflow:Writing example 0 of 144


INFO:tensorflow:Writing example 0 of 144


INFO:tensorflow:*** Example ***


INFO:tensorflow:*** Example ***


INFO:tensorflow:guid: None


...

INFO:tensorflow:input_mask: 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


INFO:tensorflow:segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


INFO:tensorflow:segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


INFO:tensorflow:label: 0 (id = 0)


INFO:tensorflow:label: 0 (id = 0)


In [0]:
test_input_fn = run_classifier.input_fn_builder(
    features=test_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=False,
    drop_remainder=False)

In [0]:
estimator.evaluate(input_fn=test_input_fn, steps=None)

INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore
C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\ops\gradients_util.py:93: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


INFO:tensorflow:Done calling model_fn.


...

INFO:tensorflow:Saving dict for global step 53: auc = 0.93365383, eval_accuracy = 0.9375, f1_score = 0.89156616, false_negatives = 3.0, false_positives = 6.0, global_step = 53, loss = 0.18877861, precision = 0.8604651, recall = 0.925, true_negatives = 98.0, true_positives = 37.0


INFO:tensorflow:Saving dict for global step 53: auc = 0.93365383, eval_accuracy = 0.9375, f1_score = 0.89156616, false_negatives = 3.0, false_positives = 6.0, global_step = 53, loss = 0.18877861, precision = 0.8604651, recall = 0.925, true_negatives = 98.0, true_positives = 37.0


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 53: output-cleaned_dataset\model.ckpt-53


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 53: output-cleaned_dataset\model.ckpt-53


{'auc': 0.93365383,
 'eval_accuracy': 0.9375,
 'f1_score': 0.89156616,
 'false_negatives': 3.0,
 'false_positives': 6.0,
 'loss': 0.18877861,
 'precision': 0.8604651,
 'recall': 0.925,
 'true_negatives': 98.0,
 'true_positives': 37.0,
 'global_step': 53}

## Make Predictions

In [0]:
def getPrediction(in_sentences):
  labels = [0, 1]
  input_examples = [run_classifier.InputExample(guid="", text_a = x, text_b = None, label = 0) for x in in_sentences] # here, "" is just a dummy label
  input_features = run_classifier.convert_examples_to_features(input_examples, label_list, MAX_SEQ_LENGTH, tokenizer)
  predict_input_fn = run_classifier.input_fn_builder(features=input_features, seq_length=MAX_SEQ_LENGTH, is_training=False, drop_remainder=False)
  predictions = estimator.predict(predict_input_fn)
  return [(sentence, prediction['probabilities'], labels[prediction['labels']]) for sentence, prediction in zip(in_sentences, predictions)]

In [0]:
pred_sentences = [
  "Issues with deliveries",
  "your online site is terrible to try to work with. Very poorly designed.",
  "Good Service",
  "Happy with your service",
    "The service has been very dependable.",
    "-Service is excellent   -Rates are beginning to increase significantly - 4.9% yearly corporate increase seems high in comparison to other carriers",
    "How I fell at the moment.  Ask me the same question at Christmas and it will probably be a 1 or 2.",
    "purolator is excellent however the pricing is slightly higher then other possibilities for shipping and they do not pick up skids in our area which a lot of our extra pieces are heavy enough that they need to be placed on a skid.",
    "The delivery guy was not at all late and was on time.",
    "We are with Purolator over 20 plus years. We have a slightly slow down with our business this year and your Sales Rep calls and needs to take back the shipping systems ESS. What can I say ? Need to ask our staff to work harder to get back the trust with Puro ?"
]

In [0]:
predictions = getPrediction(pred_sentences)

INFO:tensorflow:Writing example 0 of 10


INFO:tensorflow:Writing example 0 of 10


INFO:tensorflow:*** Example ***


INFO:tensorflow:*** Example ***


INFO:tensorflow:guid: 


...

INFO:tensorflow:Restoring parameters from output-cleaned_dataset\model.ckpt-53


INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Done running local_init_op.


INFO:tensorflow:Done running local_init_op.


In [0]:
predictions

[('Issues with deliveries',
  array([-3.0522273 , -0.04840642], dtype=float32),
  1),
 ('your online site is terrible to try to work with. Very poorly designed.',
  array([-4.0372586 , -0.01780327], dtype=float32),
  1),
 ('Good Service', array([-0.01013549, -4.596773  ], dtype=float32), 0),
 ('Happy with your service',
  array([-0.01066107, -4.5464845 ], dtype=float32),
  0),
 ('The service has been very dependable.',
  array([-0.01036275, -4.5747166 ], dtype=float32),
  0),
 ('-Service is excellent   -Rates are beginning to increase significantly - 4.9% yearly corporate increase seems high in comparison to other carriers',
  array([-2.6830924 , -0.07079966], dtype=float32),
  1),
 ('How I fell at the moment.  Ask me the same question at Christmas and it will probably be a 1 or 2.',
  array([-2.7223399 , -0.06797995], dtype=float32),
  1),
 ('purolator is excellent however the pricing is slightly higher then other possibilities for shipping and they do not pick up skids in our area wh

In [0]:
result = getPrediction(["","How I fell at the moment.  Ask me the same question at Christmas and it will probably be a 1 or 2."])

INFO:tensorflow:Writing example 0 of 2


INFO:tensorflow:Writing example 0 of 2


INFO:tensorflow:*** Example ***


INFO:tensorflow:*** Example ***


INFO:tensorflow:guid: 


...

INFO:tensorflow:Restoring parameters from output-cleaned_dataset\model.ckpt-53


INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Done running local_init_op.


INFO:tensorflow:Done running local_init_op.


In [0]:
result

[('', array([-0.9267063, -0.5039417], dtype=float32), 1),
 ('How I fell at the moment.  Ask me the same question at Christmas and it will probably be a 1 or 2.',
  array([-2.9969018 , -0.05123172], dtype=float32),
  1)]

## Create A Result File with Sentiments

### Sales Comment Sentiment Analysis

In [0]:
df_sales_oe = translate_from_french(df_sales_oe)

In [0]:
result = getPrediction(df_sales_oe['comment_modified'].values)

INFO:tensorflow:Writing example 0 of 337


INFO:tensorflow:Writing example 0 of 337


INFO:tensorflow:*** Example ***


INFO:tensorflow:*** Example ***


INFO:tensorflow:guid: 


...

INFO:tensorflow:Restoring parameters from output-cleaned_dataset\model.ckpt-53


INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Done running local_init_op.


INFO:tensorflow:Done running local_init_op.


In [0]:
series_sent = []
for item in result:
    (_,_,Sentiment) = item
    series_sent.append(Sentiment)
df_sales_oe['sentiment'] = series_sent

In [0]:
df_sales_oe.to_csv("data/sales_sentiments.csv")

### Main Comments Sentiment Analysis

In [0]:
df_main_oe = translate_from_french(df_main_oe)

In [0]:
result = getPrediction(df_main_oe['comment_modified'].values)

INFO:tensorflow:Writing example 0 of 697


INFO:tensorflow:Writing example 0 of 697


INFO:tensorflow:*** Example ***


INFO:tensorflow:*** Example ***


INFO:tensorflow:guid: 


...

INFO:tensorflow:Restoring parameters from output-cleaned_dataset\model.ckpt-53


INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Done running local_init_op.


INFO:tensorflow:Done running local_init_op.


In [0]:
series_sent = []
series_prob = []
for item in result:
    (_,Probabilities,Sentiment) = item
    series_sent.append(Sentiment)
    series_prob.append(Probabilities)
df_main_oe['sentiment'] = series_sent
df_main_oe['probabilities'] = series_prob

In [0]:
df_main_oe.to_csv("data/main_sentiments.csv")

## Get Actual Probability Values

In [0]:
import pandas as pd
df_main_oe = pd.read_csv("data/main_sentiments.csv")

In [0]:
prob0series = []
prob1series = []
for index,row in df_main_oe.iterrows():
    for item in row['probabilities'].split(" ")[1:]:
        if item == "" or item =="]":
            continue
        else:
            for item in item.split("]"):
                if item == "" or item =="]":
                    continue
                else:
                    prob1series.append(float(item))
    for item in row['probabilities'].split(" ")[:1]:
        for item in item.split("["):
            if item == "" or item =="[":
                continue
            else:
                prob0series.append(float(item))

In [0]:
df_main_oe['logit1'] = prob1series
df_main_oe['logit0'] = prob0series

In [0]:
import numpy as np
df_main_oe['probability1'] = df_main_oe.logit1.apply(np.exp)/(df_main_oe.logit1.apply(np.exp)+df_main_oe.logit0.apply(np.exp))
df_main_oe['probability0'] = df_main_oe.logit0.apply(np.exp)/(df_main_oe.logit1.apply(np.exp)+df_main_oe.logit0.apply(np.exp))

In [0]:
df_main_oe.to_csv("data/main_sentiments_with_probabilties.csv")